# Итоговое Практическое задание. Дольникова А.А.

Дано задание: "Создать приложение для задачи регрессии (одним или несколькими методами) для предсказания глубины и ширины сварного соединения."

октябрь 2023

### Импорт библиотек

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor  # регрессор ансамбля "случайного леса" из регающих деревьев и  градиентного бустинга (Gradient Boosting)
from sklearn.tree import DecisionTreeRegressor # регрессор "деревья решений"
from sklearn.neighbors import KNeighborsRegressor #регрессор "K -ближайших соседей"
from sklearn.metrics import accuracy_score # для анализа моделей
import pickle

%matplotlib inline

### Подгружаем датасет

In [30]:
df = pd.read_csv('data/ebw_data.csv')

In [31]:
pd.set_option('display.max_columns',100) # чтобы было видно все столбцы датасета
pd.set_option('display.max_rows',100) # чтобы было видно все строки датасета

In [32]:
df.head() # смотрю первые 5 строк датасета

,IW,IF,VW,FP,Depth,Width
0,47,139,4.5,80,1.60,2.54
1,47,139,4.5,80,1.62,2.50
2,47,139,4.5,80,1.68,2.60
3,47,139,4.5,80,1.58,2.52
4,45,140,4.5,80,1.76,2.48


In [33]:
df #смотрю весь датасет

,IW,IF,VW,FP,Depth,Width
0,47,139,4.5,80,1.60,2.54
1,47,139,4.5,80,1.62,2.50
2,47,139,4.5,80,1.68,2.60
3,47,139,4.5,80,1.58,2.52
4,45,140,4.5,80,1.76,2.48
5,45,140,4.5,80,1.72,2.48
6,45,140,4.5,80,1.64,2.52
7,45,140,4.5,80,1.64,2.56
8,45,140,8.0,80,1.20,2.00
9,45,140,8.0,80,1.20,1.96


# Собираем датасет

In [34]:
X = df[['IW', 'IF', 'VW', 'FP']].values # входные параметры
y1 = df['Depth'].values
y2 = df['Width'].values# выходные параметры

In [35]:
X.shape, y1.shape, y2.shape

((72, 4), (72,), (72,))

In [36]:
X_train, X_test, y1_train, y1_test = train_test_split(X, y1, test_size=0.3, random_state=0)

In [37]:
X_train, X_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.3, random_state=0)

In [38]:
X_train.shape

(50, 4)

In [39]:
#import sklearn
from sklearn.ensemble import GradientBoostingRegressor

# Метод случайного леса Random Forest Regressor

In [40]:
model_rfr1 = RandomForestRegressor()
model_rfr1_params = {
    'n_estimators' : range(10, 1000, 10),
    'criterion' : ['squared_error', 'absolute_error', 'poisson'],
    'max_depth' : range(1, 7),
    'min_samples_split' : range(20, 50, 5),
    'min_samples_leaf' : range(2, 8),
    'bootstrap' : ['True', 'False']
}

In [41]:
# обучение с параметрами по умолчанию
model_rfr1.fit(X_train, y1_train)
y1_rfr1 = model_rfr1.predict(X_test) #предсказание на основе тестовых данных
# выведем первые пять значений с помощью диапазона индексов
print(y1_rfr1[:5])
print('')
print(y1_test[:5])

[0.94942619 0.94942619 1.22159309 1.09494    1.5663202 ]

[0.92 0.94 1.28 1.2  1.4 ]


In [42]:
model_rfr2 = RandomForestRegressor()
model_rfr2_params = {
    'n_estimators' : range(10, 1000, 10),
    'criterion' : ['squared_error', 'absolute_error', 'poisson'],
    'max_depth' : range(1, 7),
    'min_samples_split' : range(20, 50, 5),
    'min_samples_leaf' : range(2, 8),
    'bootstrap' : ['True', 'False']
}

In [43]:
# обучение с параметрами по умолчанию
model_rfr2.fit(X_train, y2_train)
y2_rfr2 = model_rfr2.predict(X_test) #предсказание на основе тестовых данных
# выведем первые пять значений с помощью диапазона индексов
print(y2_rfr2[:5])
print('')
print(y2_test[:5])

[1.97358    1.97358    1.79662147 1.71432238 2.52530865]

[1.86 1.94 1.8  1.8  2.48]


In [44]:
y1_rfr1

array([0.94942619, 0.94942619, 1.22159309, 1.09494   , 1.5663202 ,
       1.22159309, 1.65615008, 1.22159309, 0.82897429, 1.29975179,
       1.22159309, 1.5663202 , 1.5663202 , 1.16324333, 0.82897429,
       1.08743333, 1.29975179, 1.22605121, 1.16324333, 1.65615008,
       1.22159309, 1.22159309])

In [45]:
y2_rfr2

array([1.97358   , 1.97358   , 1.79662147, 1.71432238, 2.52530865,
       1.79662147, 2.46551048, 1.79662147, 2.09121333, 1.76223714,
       1.79662147, 2.52530865, 2.52530865, 1.80748667, 2.09121333,
       1.81235167, 1.76223714, 1.78057992, 1.80748667, 2.46551048,
       1.79662147, 1.79662147])

# Сохраняю модель

In [46]:
pickle.dump(model_rfr1, open('model_rfr1.pkl', 'wb'))

In [50]:
pickle.dump(model_rfr2, open('model_rfr2.pkl', 'wb'))

In [51]:
model1_load=pickle.load (open('model_rfr1.pkl', 'rb'))

In [52]:
model2_load=pickle.load (open('model_rfr2.pkl', 'rb'))

In [53]:
model2_load.predict(X_test)

array([1.97358   , 1.97358   , 1.79662147, 1.71432238, 2.52530865,
       1.79662147, 2.46551048, 1.79662147, 2.09121333, 1.76223714,
       1.79662147, 2.52530865, 2.52530865, 1.80748667, 2.09121333,
       1.81235167, 1.76223714, 1.78057992, 1.80748667, 2.46551048,
       1.79662147, 1.79662147])

In [54]:
model1_load.predict(X_test)

array([0.94942619, 0.94942619, 1.22159309, 1.09494   , 1.5663202 ,
       1.22159309, 1.65615008, 1.22159309, 0.82897429, 1.29975179,
       1.22159309, 1.5663202 , 1.5663202 , 1.16324333, 0.82897429,
       1.08743333, 1.29975179, 1.22605121, 1.16324333, 1.65615008,
       1.22159309, 1.22159309])

# Дальнейшая работа ведется в Visual Studio Code